# 🚀 Mech-Exo Execution Engine Demo

This notebook demonstrates the complete execution engine workflow:
1. **Safe Mode Setup** - Configure stub mode for risk-free testing
2. **Score → Trade Flow** - Generate ideas, route orders, track fills
3. **Database Queries** - Query DuckDB for execution data
4. **JSON Logs Analysis** - Structured logging output
5. **Performance Visualization** - P&L and risk event charts

⚠️ **SAFETY**: This notebook runs in `stub` mode with simulated trading only. No real money at risk.

## 📋 Step 1: Environment Setup

In [ ]:
# 🛡️ SAFETY: Set stub mode to prevent accidental live trading
%env EXO_MODE=stub
%env PREFECT_LOGGING_LEVEL=WARNING

# Standard imports
import sys
import os
import asyncio
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, date
from pathlib import Path

# Add project to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

print("🔧 Environment configured for safe stub mode trading")
print(f"📁 Project root: {project_root}")
print(f"🛡️ Trading mode: {os.getenv('EXO_MODE', 'unknown')}")

## 💡 Step 2: Import Mech-Exo Components

In [ ]:
# Import execution engine components
from mech_exo.execution.order_router import OrderRouter
from mech_exo.execution.models import create_market_order, create_limit_order
from mech_exo.execution.fill_store import FillStore
from mech_exo.execution.safety_valve import SafetyValve
from mech_exo.risk import RiskChecker, Portfolio
from mech_exo.scoring import IdeaScorer
from tests.stubs.broker_stub import EnhancedStubBroker
from mech_exo.utils.structured_logging import configure_structured_logging, get_execution_logger, create_session_context

# Import for data analysis
from mech_exo.datasource import DataStorage

print("✅ All Mech-Exo components imported successfully")

## 🔧 Step 3: Configure Structured Logging

In [ ]:
# Configure JSON logging for this demo session
import tempfile

# Create temporary log file
log_file = tempfile.NamedTemporaryFile(mode='w', suffix='_demo.json', delete=False)
log_file.close()

# Configure structured logging
configure_structured_logging(
    log_level="INFO",
    log_file=log_file.name,
    console_output=False,  # Quiet for notebook
    json_format=True
)

# Create session context
session_context = create_session_context(
    strategy="demo_notebook",
    account_id="DEMO_ACCOUNT_001"
)

print(f"📝 Structured logging configured")
print(f"📄 Log file: {log_file.name}")
print(f"🎯 Session ID: {session_context.session_id}")

## 🏪 Step 4: Initialize Trading Components

In [ ]:
async def setup_trading_components():
    """Initialize all trading components"""
    
    # 1. Initialize StubBroker for safe testing
    broker_config = {
        'simulate_fills': True,
        'fill_delay_ms': 50,
        'initial_nav': 100000.0,
        'simulate_slippage': True,
        'price_movement': True,
        'write_to_fill_store': True
    }
    
    broker = EnhancedStubBroker(broker_config)
    connected = await broker.connect()
    
    if not connected:
        raise RuntimeError("Failed to connect to broker")
    
    # 2. Initialize Portfolio and Risk Checker
    portfolio = Portfolio(100000.0)
    
    # Mock the config loading for demo
    from unittest.mock import patch, Mock
    with patch('mech_exo.risk.checker.ConfigManager'), \
         patch('mech_exo.risk.checker.DataStorage'):
        
        risk_checker = RiskChecker(portfolio)
        # Mock risk checker to approve trades for demo
        risk_checker.check_new_position = Mock(return_value={
            'pre_trade_analysis': {
                'recommendation': 'APPROVE',
                'violations': [],
                'warnings': []
            }
        })
    
    # 3. Initialize OrderRouter with safety settings
    router_config = {
        'max_retries': 2,
        'safety': {
            'safety_mode': 'disabled',  # Safe for demo
            'max_daily_value': 50000.0
        }
    }
    
    router = OrderRouter(broker, risk_checker, router_config)
    
    # 4. Initialize FillStore for persistence
    fill_store = FillStore()  # Uses default path
    
    return broker, router, fill_store, risk_checker

# Run setup
broker, router, fill_store, risk_checker = await setup_trading_components()

# Get account info
account_info = await broker.get_account_info()
print(f"✅ Trading components initialized")
print(f"🏦 Account: {account_info['account_id']}")
print(f"💰 NAV: ${account_info['netliquidation']:,.0f}")
print(f"💵 Cash: ${account_info['totalcashvalue']:,.0f}")

## 🎯 Step 5: Generate Investment Ideas (Score)

In [ ]:
# For demo purposes, we'll create mock investment ideas
# In production, this would come from the IdeaScorer

demo_ideas = [
    {'symbol': 'AAPL', 'score': 0.85, 'signal_strength': 1.2, 'strategy': 'momentum'},
    {'symbol': 'GOOGL', 'score': 0.78, 'signal_strength': 1.1, 'strategy': 'value'},
    {'symbol': 'MSFT', 'score': 0.72, 'signal_strength': 0.9, 'strategy': 'quality'},
    {'symbol': 'TSLA', 'score': 0.68, 'signal_strength': 1.3, 'strategy': 'growth'},
    {'symbol': 'SPY', 'score': 0.65, 'signal_strength': 0.8, 'strategy': 'beta'}
]

ideas_df = pd.DataFrame(demo_ideas)
print("📊 Investment Ideas Generated:")
print(ideas_df.to_string(index=False))

# Select top 3 ideas for trading
top_ideas = ideas_df.head(3)
print(f"\n🏆 Selected {len(top_ideas)} top ideas for execution")

## 📈 Step 6: Execute Trades (Trade Flow)

In [ ]:
async def execute_trading_session(ideas_df, router):
    """Execute a complete trading session"""
    
    orders_placed = []
    routing_results = []
    
    print("🚀 Starting Trading Session...\n")
    
    for idx, idea in ideas_df.iterrows():
        symbol = idea['symbol']
        strategy = idea['strategy']
        signal_strength = idea['signal_strength']
        
        # Create order based on signal strength
        if signal_strength > 1.0:
            # Strong signal -> market order
            order = create_market_order(
                symbol=symbol,
                quantity=100,  # Fixed size for demo
                strategy=strategy
            )
            order_type = "Market"
        else:
            # Weak signal -> limit order
            market_price = broker.get_market_price(symbol) or 150.0
            limit_price = market_price * 0.99  # 1% below market
            order = create_limit_order(
                symbol=symbol,
                quantity=100,
                limit_price=limit_price,
                strategy=strategy
            )
            order_type = f"Limit @ ${limit_price:.2f}"
        
        print(f"📤 {idx+1}. Routing {symbol} ({strategy}) - {order_type}")
        
        # Route order through full workflow
        result = await router.route_order(order)
        
        orders_placed.append(order)
        routing_results.append(result)
        
        if result.decision.value == 'APPROVE':
            print(f"   ✅ Approved: {result.routing_notes}")
        else:
            print(f"   ❌ Rejected: {result.rejection_reason}")
        
        # Small delay for realistic timing
        await asyncio.sleep(0.1)
    
    # Wait for fills
    print("\n⏳ Waiting for fills...")
    await asyncio.sleep(0.5)
    
    return orders_placed, routing_results

# Execute trading session
orders, results = await execute_trading_session(top_ideas, router)

# Summary
approved = sum(1 for r in results if r.decision.value == 'APPROVE')
print(f"\n📊 Session Summary:")
print(f"   Orders placed: {len(orders)}")
print(f"   Approved: {approved}")
print(f"   Rejected: {len(orders) - approved}")

## 💾 Step 7: Query Execution Database

In [ ]:
# Query today's fills from DuckDB
today = date.today()
fills = fill_store.get_fills()

if fills:
    print(f"📊 Today's Fills ({len(fills)} total):")
    
    # Convert to DataFrame for analysis
    fill_data = []
    for fill in fills:
        fill_data.append({
            'Symbol': fill.symbol,
            'Quantity': fill.quantity,
            'Price': f"${fill.price:.2f}",
            'Value': f"${abs(fill.quantity * fill.price):,.0f}",
            'Commission': f"${fill.commission:.2f}",
            'Strategy': fill.strategy,
            'Time': fill.fill_time.strftime('%H:%M:%S')
        })
    
    fills_df = pd.DataFrame(fill_data)
    print(fills_df.to_string(index=False))
    
    # Get daily metrics
    daily_metrics = fill_store.get_daily_metrics(today)
    print(f"\n📈 Daily Execution Metrics:")
    print(f"   Total fills: {daily_metrics['fills']['total_fills']}")
    print(f"   Total value: ${daily_metrics['fills']['total_value']:,.0f}")
    print(f"   Symbols traded: {daily_metrics['fills']['symbols_traded']}")
    print(f"   Avg commission: ${daily_metrics['fills']['avg_commission']:.2f}")
    
else:
    print("📭 No fills found for today")

# Query current positions
positions = await broker.get_positions()
if positions:
    print(f"\n🏦 Current Positions ({len(positions)} total):")
    pos_df = pd.DataFrame(positions)
    if not pos_df.empty:
        print(pos_df[['symbol', 'position', 'market_value', 'unrealized_pnl']].to_string(index=False))
else:
    print("\n📭 No open positions")

## 📄 Step 8: Analyze JSON Logs

In [ ]:
# Read and analyze structured logs
def analyze_json_logs(log_file_path):
    """Parse and analyze JSON logs from the trading session"""
    
    try:
        with open(log_file_path, 'r') as f:
            log_lines = f.readlines()
        
        if not log_lines:
            print("📭 No log entries found")
            return None, None
        
        # Parse JSON logs
        events = []
        for line in log_lines:
            try:
                event = json.loads(line.strip())
                events.append(event)
            except json.JSONDecodeError:
                continue
        
        if not events:
            print("📭 No valid JSON events found")
            return None, None
        
        # Categorize events
        order_events = []
        fill_events = []
        performance_events = []
        safety_events = []
        
        for event in events:
            extra = event.get('extra', {})
            event_type = extra.get('event_type', '')
            
            if event_type.startswith('order.'):
                order_events.append(event)
            elif event_type.startswith('execution.fill'):
                fill_events.append(event)
            elif event_type.startswith('performance.'):
                performance_events.append(event)
            elif event_type.startswith('safety.'):
                safety_events.append(event)
        
        print(f"📊 JSON Log Analysis ({len(events)} total events):")
        print(f"   📋 Order events: {len(order_events)}")
        print(f"   💰 Fill events: {len(fill_events)}")
        print(f"   ⚡ Performance events: {len(performance_events)}")
        print(f"   🛡️ Safety events: {len(safety_events)}")
        
        # Show sample events
        if fill_events:
            print(f"\n📄 Sample Fill Event:")
            sample_fill = fill_events[0]
            extra = sample_fill.get('extra', {})
            print(f"   Symbol: {extra.get('symbol')}")
            print(f"   Quantity: {extra.get('quantity')}")
            print(f"   Price: ${extra.get('price'):.2f}")
            print(f"   Commission: ${extra.get('commission'):.2f}")
            print(f"   Strategy: {extra.get('strategy')}")
        
        if performance_events:
            print(f"\n⚡ Sample Performance Event:")
            sample_perf = performance_events[0]
            extra = sample_perf.get('extra', {})
            print(f"   Metric: {extra.get('metric_name')}")
            print(f"   Value: {extra.get('metric_value'):.2f} {extra.get('metric_unit')}")
            print(f"   Operation: {extra.get('operation', 'N/A')}")
        
        return events, {
            'orders': order_events,
            'fills': fill_events,
            'performance': performance_events,
            'safety': safety_events
        }
        
    except Exception as e:
        print(f"❌ Error analyzing logs: {e}")
        return None, None

# Analyze logs from this session
all_events, categorized_events = analyze_json_logs(log_file.name)

## 📊 Step 9: Performance Visualization

In [ ]:
# Set up plotting style
plt.style.use('seaborn-v0_8')
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('🚀 Mech-Exo Execution Engine Demo Results', fontsize=16, fontweight='bold')

# 1. Portfolio Value Chart
account_info = await broker.get_account_info()
current_nav = account_info['netliquidation']
initial_nav = 100000

navs = [initial_nav, current_nav]
times = ['Start', 'Current']

ax1.plot(times, navs, marker='o', linewidth=3, markersize=8, color='darkgreen')
ax1.set_title('💰 Portfolio NAV', fontweight='bold')
ax1.set_ylabel('NAV ($)')
ax1.grid(True, alpha=0.3)
for i, nav in enumerate(navs):
    ax1.annotate(f'${nav:,.0f}', (i, nav), textcoords="offset points", xytext=(0,10), ha='center')

# 2. Fill Volume by Symbol
if fills:
    symbols = [f.symbol for f in fills]
    volumes = [abs(f.quantity * f.price) for f in fills]
    
    symbol_volumes = {}
    for symbol, volume in zip(symbols, volumes):
        symbol_volumes[symbol] = symbol_volumes.get(symbol, 0) + volume
    
    ax2.bar(symbol_volumes.keys(), symbol_volumes.values(), color='steelblue', alpha=0.7)
    ax2.set_title('📊 Trading Volume by Symbol', fontweight='bold')
    ax2.set_ylabel('Volume ($)')
    ax2.tick_params(axis='x', rotation=45)
else:
    ax2.text(0.5, 0.5, 'No fills to display', ha='center', va='center', transform=ax2.transAxes)
    ax2.set_title('📊 Trading Volume by Symbol', fontweight='bold')

# 3. Event Timeline
if categorized_events and categorized_events['performance']:
    perf_events = categorized_events['performance']
    metrics = []
    values = []
    
    for event in perf_events[:10]:  # Top 10 events
        extra = event.get('extra', {})
        metric_name = extra.get('metric_name', 'unknown')
        metric_value = extra.get('metric_value', 0)
        
        if 'duration' in metric_name:
            metrics.append(metric_name.replace('_duration', '').replace('_', ' ').title())
            values.append(metric_value)
    
    if metrics:
        ax3.barh(metrics, values, color='orange', alpha=0.7)
        ax3.set_title('⚡ Execution Performance (ms)', fontweight='bold')
        ax3.set_xlabel('Duration (ms)')
    else:
        ax3.text(0.5, 0.5, 'No performance metrics', ha='center', va='center', transform=ax3.transAxes)
        ax3.set_title('⚡ Execution Performance', fontweight='bold')
else:
    ax3.text(0.5, 0.5, 'No performance data', ha='center', va='center', transform=ax3.transAxes)
    ax3.set_title('⚡ Execution Performance', fontweight='bold')

# 4. Trade Distribution
if results:
    approved = sum(1 for r in results if r.decision.value == 'APPROVE')
    rejected = len(results) - approved
    
    labels = ['Approved', 'Rejected']
    sizes = [approved, rejected]
    colors = ['lightgreen', 'lightcoral']
    
    ax4.pie(sizes, labels=labels, colors=colors, autopct='%1.0f%%', startangle=90)
    ax4.set_title('📈 Trade Approval Rate', fontweight='bold')
else:
    ax4.text(0.5, 0.5, 'No trades executed', ha='center', va='center', transform=ax4.transAxes)
    ax4.set_title('📈 Trade Approval Rate', fontweight='bold')

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\n📈 Session Performance Summary:")
print(f"   Initial NAV: ${initial_nav:,.0f}")
print(f"   Current NAV: ${current_nav:,.0f}")
print(f"   P&L: ${current_nav - initial_nav:+,.0f} ({((current_nav/initial_nav-1)*100):+.2f}%)")
print(f"   Orders placed: {len(orders)}")
print(f"   Fills received: {len(fills) if fills else 0}")
print(f"   JSON events logged: {len(all_events) if all_events else 0}")

## 🔗 Step 10: Prefect Integration Preview

In production, this execution flow would be orchestrated by Prefect. Here's how it connects:

In [ ]:
# Mock Prefect flow integration
print("🔗 Prefect Integration Points:")
print("")
print("📋 1. Daily Flow Orchestration:")
print("   ├── 06:00 - Data collection (Phase P1)")
print("   ├── 06:30 - Idea scoring (Phase P2)")
print("   ├── 07:00 - Position sizing (Phase P3)")
print("   ├── 09:30 - Order execution (Phase P4) ← This demo")
print("   └── 16:00 - Daily reporting (Phase P5) ← Next phase")
print("")
print("🚀 Production Command:")
print("   export EXO_MODE=paper  # or 'live'")
print("   prefect deployment run 'daily-trading-flow/production'")
print("")
print("📊 Monitoring:")
print("   - Prefect UI: http://localhost:4200")
print("   - JSON logs: Real-time execution monitoring")
print("   - DuckDB: Complete audit trail")
print("   - Safety valve: Multi-layer protection")

## 🧹 Step 11: Cleanup

In [ ]:
# Cleanup resources
await broker.disconnect()
fill_store.close()
risk_checker.close()

print("✅ Demo completed successfully!")
print(f"📄 Log file preserved: {log_file.name}")
print("")
print("🎯 Next Steps:")
print("   1. Review logs for detailed execution flow")
print("   2. Explore data in DuckDB execution database")
print("   3. Configure IB Gateway for paper trading")
print("   4. Set up Prefect for automated execution")
print("")
print("⚠️  Remember: Always test with EXO_MODE=paper before going live!")

---

## 🎉 Demo Complete!

You have successfully run the complete Mech-Exo execution engine workflow:

✅ **Safe Environment**: Stub mode ensures no real money risk  
✅ **End-to-End Flow**: Ideas → Orders → Fills → Database → Logs  
✅ **Performance Tracking**: JSON logs with structured monitoring  
✅ **Data Persistence**: DuckDB storage for audit trails  
✅ **Visualization**: Charts showing execution performance  

### 🚀 Ready for Production

The execution engine is production-ready with:
- Multi-mode operation (stub/paper/live)
- Comprehensive safety controls
- Real-time risk monitoring
- Structured logging & metrics
- Complete audit trails

**Next Phase**: P5 Reporting & Dashboard for live monitoring and analytics.